In [1]:
import yaml
import wandb

config_path = "configs/base_config.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# wandb.init(project=config['wandb']['project'],
#            entity=config['wandb']['entity'],
#            config=config)

In [2]:
import torchreid
datamanager = torchreid.data.ImageDataManager(
    root            = config['data']['root'],
    sources         = config['data']['sources'],
    targets         = config['data']['targets'],
    height          = config['data']['height'],
    width           = config['data']['width'],
    batch_size_train= config['data']['batch_size'],
    batch_size_test = config['data']['batch_size'],
    transforms      = config['data']['transforms']
)

/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [3]:
model = torchreid.models.build_model(
    name        = config['model']['name'],
    num_classes = 2000 if 'tango2000' in config['data']['sources'] else datamanager.num_train_pids, #2000 for Tango
    loss        = config['loss']['name'],
    pretrained  = config['model']['pretrained'],
    pooling     = config['model']['final_pooling'],
    **config['custom']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim=config['train']['optim'],
    lr=config['train']['lr']
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    **config['train']['lr_scheduler'],
)

In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [6]:
engine = torchreid.engine.ProxyAwareEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    margin=config['loss']['proxyAware']['margin'],
    alpha=config['loss']['proxyAware']['alpha'],
    # label_smooth=config['loss']['softmax']['label_smooth'],
    # logger=wandb
)

In [7]:
engine.run(
    save_dir=f"log/{config['model']['name']}_{config['data']['sources'][0]}",
    max_epoch=60,
    eval_freq=5,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training
epoch: [1/60][50/101]	time 0.152 (0.170)	data 0.000 (0.006)	eta 0:17:02	loss 9.7635 (10.5181)	acc 0.0000 (0.0625)	lr 0.001000
epoch: [1/60][100/101]	time 0.150 (0.161)	data 0.000 (0.003)	eta 0:16:01	loss 9.2014 (10.0023)	acc 0.7812 (0.1484)	lr 0.001000
epoch: [2/60][50/101]	time 0.153 (0.156)	data 0.000 (0.004)	eta 0:15:20	loss 8.9040 (8.9758)	acc 0.0000 (0.1406)	lr 0.001000
epoch: [2/60][100/101]	time 0.154 (0.154)	data 0.000 (0.002)	eta 0:15:02	loss 8.7813 (8.8755)	acc 0.0000 (0.1484)	lr 0.001000
epoch: [3/60][50/101]	time 0.153 (0.156)	data 0.000 (0.004)	eta 0:15:08	loss 8.4431 (8.5444)	acc 0.7812 (0.2812)	lr 0.001000
epoch: [3/60][100/101]	time 0.153 (0.155)	data 0.000 (0.002)	eta 0:14:51	loss 8.4119 (8.5067)	acc 0.0000 (0.1875)	lr 0.001000
epoch: [4/60][50/101]	time 0.152 (0.156)	data 0.000 (0.004)	eta 0:14:52	loss 8.2622 (8.2853)	acc 0.0000 (0.2812)	lr 0.001000
epoch: [4/60][100/101]	time 0.150 (0.154)	data 0.000 (0.002)	eta 0:14:32	loss 8.1903 (8.2588)	acc 0.00

In [ ]:
wandb.finish()